# How many max features?

In [ ]:
# Run AWS CLI configuration in the terminal as `aws configure` or here in the notebook
# Note: This is typically done in the terminal, but you can also run it in a notebook cell
# Uncomment the next line to run it in a notebook cell (not recommended for production use)
# !aws configure

In [1]:
# Step 2: Set up the MLflow tracking server
from dotenv import load_dotenv
load_dotenv()  # This loads environment variables from .env

import os
import mlflow

mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")

2025/07/11 04:37:17 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-2025/355715601687537573', creation_time=1752233837464, experiment_id='355715601687537573', last_update_time=1752233837464, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [4]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        # Create the 'results' directory if it doesn't exist
        os.makedirs("results/notebook_4", exist_ok=True)
        plt.savefig("results/notebook_4/confusion_matrix.png")
        mlflow.log_artifact("results/notebook_4/confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/07/11 04:39:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 04:39:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573/runs/5de8e7a6ee2f4b43b65dc8353cb77061
🧪 View experiment at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573


2025/07/11 04:40:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 04:40:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573/runs/da2ac698b67840f0ab19fb5a013c1da5
🧪 View experiment at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573


2025/07/11 04:40:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 04:40:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573/runs/6614ece5930f4af3bb24265e7e9ba98b
🧪 View experiment at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573


2025/07/11 04:42:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 04:42:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573/runs/560ced21bfd54cbcaf2f0550a7f53410
🧪 View experiment at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573


2025/07/11 04:43:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 04:43:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573/runs/c2150f12cf064b0abcd81ef09f5b2046
🧪 View experiment at: http://ec2-44-249-137-23.us-west-2.compute.amazonaws.com:5000/#/experiments/355715601687537573
